In [1]:
#!pip install --upgrade google-cloud-storage

In [2]:
import os
import psycopg2
import pandas as pd
from google.cloud import storage

# Extract

In [3]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "etl-acckey.json"
client = storage.Client()

In [4]:
bucket_name = 'etl_bucket1292021'
# new_bucket = client.create_bucket(bucket_name)

In [5]:
# Retrieve an existing bucket
# https://console.cloud.google.com/storage/browser/[bucket-id]/
bucket = client.get_bucket(bucket_name)
blob = bucket.blob('Canada.xlsx')

In [6]:
with open('Canada.xlsx','wb') as f:
    client.download_blob_to_file(blob, f)

# Transform

In [7]:
df = pd.read_excel("Canada.xlsx",skiprows=20,skipfooter=2,sheet_name='Canada by Citizenship')
df.head()

,Type,Coverage,OdName,AREA,AreaName,REG,RegName,DEV,DevName,1980,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
0,Immigrants,Foreigners,Afghanistan,935,Asia,5501,Southern Asia,902,Developing regions,16,...,2978,3436,3009,2652,2111,1746,1758,2203,2635,2004
1,Immigrants,Foreigners,Albania,908,Europe,925,Southern Europe,901,Developed regions,1,...,1450,1223,856,702,560,716,561,539,620,603
2,Immigrants,Foreigners,Algeria,903,Africa,912,Northern Africa,902,Developing regions,80,...,3616,3626,4807,3623,4005,5393,4752,4325,3774,4331
3,Immigrants,Foreigners,American Samoa,909,Oceania,957,Polynesia,902,Developing regions,0,...,0,0,1,0,0,0,0,0,0,0
4,Immigrants,Foreigners,Andorra,908,Europe,925,Southern Europe,901,Developed regions,0,...,0,0,1,1,0,0,0,0,1,1


In [8]:
df.rename(columns={'OdName':'Country','AreaName':'Continent','RegName':'Region'},inplace=True)

In [9]:
df.drop(['Type','Coverage','AREA','REG','DEV'],axis=1,inplace=True)

In [10]:
#df.set_index('Country',inplace=True)

In [11]:
df.columns = df.columns.astype(str)
years = list(map(str,range(1980,2014)))
df['Total'] = df[years].sum(axis=1)

In [12]:
#df.reset_index(inplace=True)

Postgres doesn't allow to name integers as column names. So I am renaming all the columns with integer
names so that they start with a underscore.

In [13]:
df.rename(columns={'1980':'_1980','1981':'_1981','1982':'_1982','1983':'_1983',
                  '1984':'_1984','1985':'_1985','1986':'_1986','1987':'_1987',
                  '1988':'_1988','1989':'_1989','1990':'_1990','1991':'_1991',
                  '1992':'_1992','1993':'_1993','1994':'_1994','1995':'_1995',
                  '1996':'_1996','1997':'_1997','1998':'_1998','1999':'_1999',
                  '2000':'_2000','2001':'_2001','2002':'_2002','2003':'_2003',
                   '2004':'_2004','2005':'_2005','2006':'_2006','2007':'_2007',
                  '2008':'_2008','2009':'_2009','2010':'_2010','2011':'_2011',
                  '2012':'_2012','2013':'_2013'},inplace=True)

Two of the countries in this dataset contain ',' in their country name. Therefore when using ',' as separator
postgres copy command will show error. Replacing ',' with '-' in the names of these two countries.

In [14]:
df['Country'] = df['Country'].str.replace(',','-')

Save the transformed dataset in a new CSV file.

In [15]:
df.to_csv("canada_dataset_cleaned.csv",index=False)

# Load

Now create a table in postgres and copy the csv data of above file in that table.

Problem is I will have to create a table in postgres by first specifying its schema. Need to enlist all the column 
names along with their data types. Now what if above transformations need to change then I will have to create a new table
by specifying another updated schema again. (DRAWBACK OF ETL)

In [16]:
conn = psycopg2.connect("dbname=postgres user=postgres password=12345678")
cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS Canada;")
cur.execute("CREATE TABLE Canada( \
            Country VARCHAR, \
            Continent VARCHAR, \
            Region VARCHAR, \
            DevName VARCHAR, \
            _1980 INTEGER, \
            _1981 INTEGER, \
            _1982 INTEGER, \
            _1983 INTEGER, \
            _1984 INTEGER, \
            _1985 INTEGER, \
            _1986 INTEGER, \
            _1987 INTEGER, \
            _1988 INTEGER, \
            _1989 INTEGER, \
            _1990 INTEGER, \
            _1991 INTEGER, \
            _1992 INTEGER, \
            _1993 INTEGER, \
            _1994 INTEGER, \
            _1995 INTEGER, \
            _1996 INTEGER, \
            _1997 INTEGER, \
            _1998 INTEGER, \
            _1999 INTEGER, \
            _2000 INTEGER, \
            _2001 INTEGER, \
            _2002 INTEGER, \
            _2003 INTEGER, \
            _2004 INTEGER, \
            _2005 INTEGER, \
            _2006 INTEGER, \
            _2007 INTEGER, \
            _2008 INTEGER, \
            _2009 INTEGER, \
            _2010 INTEGER, \
            _2011 INTEGER, \
            _2012 INTEGER, \
            _2013 INTEGER, \
            Total INTEGER);")

In [17]:
with open("canada_dataset_cleaned.csv",'r') as csv_file:
    next(csv_file) # to skip the header - because column names have already been defined in PostgreSQL table
    cur.copy_from(csv_file, 'Canada', sep=',')
    
conn.commit()

In [18]:
cur.close()
conn.close()